Zach Kelly

## research question/interests

The objective behind this repository is to train a machine learning model using a set of fighter statistics to predict score outcomes of future ufc bouts. 

The desired learning outcomes are the following:
- learn how to handle large data sets with python.
- further learn how to formulate problems concerning machine learning.  
- further understand the challenges of training a successful machine learning model. 

## import raw data with pandas

In [ ]:
import numpy as np
import pandas as pd

r_data = pd.read_csv("raw_data.csv")